In [1]:
#include <memory>
#include <iostream>
#include <map>
#include <string>

In [2]:
class Resource
{
public:
    int number;
    
	Resource(int n) {
        number = n;
        std::cout << "Resource acquired\n"; 
    }
    
	~Resource() { 
        std::cout << "Resource destroyed\n"; 
    }
    
    friend std::ostream& operator<<(std::ostream& out, const Resource& res)
	{
		out << "I am a resource\n";
		return out;
	}
};

# 1 basic operation

## 1.a How to construct a std::unique_ptr

method1: use new

In [3]:
std::unique_ptr<Resource> res1{new Resource(5)}

Resource acquired


method2: use std:make_unique

In [4]:
std::unique_ptr<Resource> res2 = std::make_unique<Resource>(5);

Resource acquired


In [5]:
{
    std::unique_ptr<Resource> res3 = std::make_unique<Resource>(5);
    
    //dereference the pointer
    std::cout << *res3 << std::endl;
}

Resource acquired
I am a resource

Resource destroyed


## 1.b how std::unique_ptr work and the lifetime of smart_pointer and  the Resource it point to.

In [6]:
int do_something1(){
    
    std::unique_ptr<Resource> res1 =  std::make_unique<Resource>(2);
    
    std::cout << "exiting " << std::endl;
    return 0;
}; //res1 go out of scope, and then release Resource

In [7]:
int a = do_something1();

Resource acquired
exiting 
Resource destroyed


## 1.c  One resource, one std::unique_ptr: copy construct and copy assgnment disabled

In [8]:
int test_rule_of_five(){
    std::unique_ptr<Resource> p1{ new Resource{22} }; // Resource created here
	std::unique_ptr<Resource> p2{}; // Start as nullptr
 
	std::cout << "p1 is " << (static_cast<bool>(p1) ? "not null\n" : "null\n");
	std::cout << "p2 is " << (static_cast<bool>(p2) ? "not null\n" : "null\n");
 
    // //Copy construction is not allowed
    //unique_ptr<MyObject> p3(p1);
	
    // res2 = res1; // Won't compile: copy assignment is disabled
    
    // the output of std:move is a rvalue, the call the move assignment operator rather than copy assignment which is disabled 
	p2 = std::move(p1); // res2 assumes ownership, res1 is set to null
 
	std::cout << "Ownership transferred\n";
 
	std::cout << "p1 is " << (static_cast<bool>(p1) ? "not null\n" : "null\n");
	std::cout << "p2 is " << (static_cast<bool>(p2) ? "not null\n" : "null\n");
     
    std::cout << *p2 << std::endl;
    
	return 0;
};

In [9]:
int b = test_rule_of_five();

Resource acquired
p1 is not null
p2 is null
Ownership transferred
p1 is null
p2 is not null
I am a resource

Resource destroyed


## 1-d: Passing std::unique_ptr to a function

Moving a unique_ptr vs. using get()?

In [10]:
void takeOwnership(std::unique_ptr<Resource> res){
     if (res)
          std::cout << *res << '\n';
}; // the Resource is destroyed here


In [11]:
int test_unique_pointer_as_function_parameter1(){
    
    auto ptr{ std::make_unique<Resource>(111)};
 
//    takeOwnership(ptr); // This doesn't work, need to use move semantics
    takeOwnership(std::move(ptr)); // ok: use move semantics
 
    std::cout << "Ending program\n";
 
    return 0;
};

int d1 = test_unique_pointer_as_function_parameter1();

Resource acquired
I am a resource

Resource destroyed
Ending program


In [12]:
// The function only uses the resource, so we'll accept a pointer to the resource, not a reference to the whole std::unique_ptr<Resource>
void justUseResource(Resource* res){
    if (res)
          std::cout << *res << '\n';
};

In [36]:
int test_unique_pointer_as_function_parameter2(){
    
    auto ptr{ std::make_unique<Resource>(111)};
 
    justUseResource(ptr.get()); // ok
 
    std::cout << "Ending program\n";
 
    return 0;
};

int d2 = test_unique_pointer_as_function_parameter2();

Resource acquired
I am a resource

Ending program
Resource destroyed


## 1-e: Returning std::unique_ptr from a function

**return by value, raw pointer to Resource, unique_ptr reference??**

### case1: return by value: be carefully

In [14]:
std::unique_ptr<Resource> createUniquePointer(){
    // method 1:
    /*create a local unique pointer
    std::unique_ptr<Resource> p1(new Resource(111));
    
    Just wanted to add one more point that returning by value should be the default choice here because a named value in the return statement in the worst case, i.e. without elisions in C++11, C++14 and C++17 is treated as an rvalue. So for example the following function compiles with the -fno-elide-constructors flag
    return p1; // p1 will surrender ownership
    */
    
    //method2
    //return std:move(p1);
    
    //method3
    return std::make_unique<Resource>(111); //call the implicit std:move senmantics 
    
};


In [15]:
void test_unique_pointer_as_function_return(){
    std::unique_ptr<Resource> a1; //unique_ptr points to nothing
    a1=createUniquePointer(); //now a1 owns the object, here call the move assignment operator
    
    std::cout << *a1 << std::endl;
    std::cout << "program end " << std::endl;
};

test_unique_pointer_as_function_return();

Resource acquired
I am a resource

program end 
Resource destroyed


### case2: return by smart pointer reference: bad idea,

 it is indicative of bad design. Having a non-const reference means that you can steal the resource or replace it without having to offer separate methods.
 
 https://stackoverflow.com/questions/48863969/can-i-pass-a-unique-ptrs-reference-to-a-function

In [16]:
class TmContainer {
public:
    TmContainer() {
        // Create some sort of complex object on heap and store unique_ptr to it
        m_time = std::unique_ptr<tm>(new tm());
        // Store something meaningful in its fields
        m_time->tm_year = 42;
    }

    std::unique_ptr<tm>& time() { return m_time; }

private:
    std::unique_ptr<tm> m_time;
};

~~~
// Just create a handle to the managed object
auto& tm_ptr = tm_container.time();
do_something_to_tm(*tm_ptr);

// Steal the resource
std::unique_ptr<TmContainer> other_tm_ptr = std::move(tm_ptr);

// Replace the managed object with another one
tm_ptr = std::make_unique<TmContainer>;
~~~

### case3: return raw pointer of Resource

You should only return and accept smart pointers when you care about their ownership semantics. If you only care about what they're pointing to, you should instead return a reference or a raw pointer.

https://stackoverflow.com/questions/43400850/return-a-unique-ptr-by-reference

So I need the users of this class to get access to the order unique_ptr if found. However I'm not going to move the object out of the vector so I'm returning the unique_ptr as const ref. My implementation of the find_order method:

~~~
class order_collection {
    typedef std::unique_ptr<order> ord_ptr;
    typedef std::vector<ord_ptr> ord_ptr_vec;
    ord_ptr_vec orders;
    ord_ptr null_unique;
public:
    ...
    const ord_ptr & find_order(std::string);
    ....
~~~
        
~~~
const order_collection::ord_ptr & order_collection::find_order(std::string id) {
    auto it = std::find_if(orders.begin(),orders.end(),
                           [&](const order_collection::ord_ptr & sptr) {
                               return sptr->getId() == id;
                           });

    if (it == orders.end())
        return null_unique; // can't return nullptr here
    return *it;
}
~~~

Since I'm returning by reference I can't return a nullptr. If I try to do so, I get warning : returning reference to a temporary. And if nothing is found the program crashes. So I added a unique_ptr<order> member variable called null_unique and I return it when find doesn't find an order. This solves the problem and warning is gone and doesn't crash when no order is found.

However I'm doubting my solution as it make my class ugly. Is this the best practice for handling this situation?

Since you're returning a dummy null_unique, it is clear that the caller of the method doesn't care about the ownership semantics. You can also have a null state: you should therefore return a raw pointer:

~~~
order* order_collection::find_order(std::string id) {
    auto it = std::find_if(orders.begin(),orders.end(),
                           [&](const order_collection::ord_ptr & sptr) {
                               return sptr->getId() == id;
                           });

    if (it == orders.end())
        return nullptr;
    return it->get();
}
~~~

It doesn't really make sense to return a unique_ptr here, reference or otherwise. A unique_ptr implies ownership over the object, and those aren't really the semantics being conveyed by this code.

As suggested in the comments, simply returning a raw pointer is fine here, provided that your Project Design explicitly prohibits you or anyone on your team from calling delete or delete[] outside the context of the destructor of a Resource-owning object.

> +1 for the raw pointer solution. I will also add that it's very common to return iterators to collections for find-like operations. You'd return the end iterator on not found. Perhaps with C++17 people will start moving towards a std::optional approach, though

## 1-f: unique_pointer in dict

In [17]:
using ResourceMap = std::map<std::string, std::unique_ptr<Resource>>;

In [18]:
ResourceMap myMap;
myMap["000001"] = std::make_unique<Resource>(11);
myMap["000002"] = std::make_unique<Resource>(21);
myMap["000003"] = std::make_unique<Resource>(31);

Resource acquired
Resource acquired
Resource acquired


In [19]:
using RMI = std::map<std::string, std::unique_ptr<Resource>>::iterator;

In [20]:
RMI it = myMap.find("000002");

In [21]:
std::cout << it -> second -> number << std::endl;

21


In [24]:
{
    //std::unique_ptr<Resource> a_copy = std::move(it -> second);
    std::cout << it -> second -> number << std::endl;
}

21


In [25]:
for(auto i = myMap.begin(); i!=myMap.end(); i++){
    std::cout <<i->first << ":" << i->second -> number << std::endl;
}

000001:11
000002:21
000003:31


In [26]:
{
    std::unique_ptr<Resource> a_copy = std::move(it -> second);
    std::cout << a_copy -> number << std::endl;
}

21
Resource destroyed


In [ ]:
//for(auto i = myMap.begin(); i!=myMap.end(); i++){
//    std::cout <<i->first << ":" << i->second -> number << std::endl;
//}

why ? because the value of the 000002 is nullptr because std::move

# 2 understand unique_ptr further

**std::unique_ptr<T,Deleter>::get**

Returns a pointer to the managed object or nullptr if no object is owned.

In [27]:
std::unique_ptr<Resource> up(new Resource{111});
std::cout << up->number << std::endl;

Resource acquired
111


In [30]:
Resource* p = up.get();
std::cout << *p << std::endl;
std::cout << p -> number << std::endl;

I am a resource

111


In [32]:
p -> number = 22;
std::cout << *p << std::endl;
std::cout << p -> number << std::endl;

I am a resource

22


In [33]:
std::cout << up->number << std::endl;

22


***see***: the research subject of smart pointer is the pointer itself, which mean we can create a pointer which has many attributes we desired. such as the unique_ptr can not be copied constructed and copy assigned.

> std::unique_ptr is a smart pointer that owns and manages another object through a pointer and disposes of that object when the unique_ptr goes out of scope.

The object is disposed of, using the associated deleter when either of the following happens:

the managing unique_ptr object is destroyed

the managing unique_ptr object is assigned another pointer via operator= or reset()